### Projeto 2-1 Visão Computacional
Elisa Malzoni

In [ ]:
%reset -f
%matplotlib inline

import cv2 as cv2
import numpy as np
import matplotlib.pyplot as plt

import time

## Aula 09 - Good features to Track

In [ ]:
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)

# Parametriza a funcao do OpenCV
dt_params = dict( maxCorners = 100,
           qualityLevel = 0.3,
           minDistance = 7,
           blockSize = 7 )

color = np.random.randint(0,255,(100,3))

# Parametriza o Lucas-Kanade
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))


ret, previous = captura.read()

previous_gray = cv2.cvtColor(previous, cv2.COLOR_RGB2GRAY)
p0 = cv2.goodFeaturesToTrack(previous_gray, mask = None, **dt_params)
mask = np.zeros_like(previous)

fomv = 0

while(1):
    
    ret, actual = captura.read()
    actual_gray = cv2.cvtColor(actual, cv2.COLOR_RGB2GRAY)
    
    # Calcula Fluxo Otico
    p1, st, err = cv2.calcOpticalFlowPyrLK(previous_gray, actual_gray, p0, None, **lk_params)
        
    # Seleciona somente os melhores pontos
    good_new = p1[st==1]
    good_old = p0[st==1]
       

    # Desenha as trilhas para cada ponto em p1 e p0
    for i,(new, old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), [0,0,255], 2)
        actual = cv2.circle(actual,(a,b),5,color[i].tolist(),-1)
    

    # Calcula fluxo otico medio
    sub = np.subtract(good_new, good_old)
    fom = np.mean(sub, axis=0) 
    fom = (-1)*fom
    fomv += fom
#     print(fomv)
    
    frame = cv2.add(actual, mask)
    framebw = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    rows, cols = framebw.shape
    
    tx = (int(fomv[0]))
    ty = (int(fomv[1]))
    
    M = np.float32([[1,0,tx],
                    [0,1,ty]])
    
    framet = cv2.warpAffine(frame, M,(cols,rows))
    
    cv2.imshow("Video", framet)
    
    # Atualiza a imagem anterior com a imagem atual e copia os pontos.
    previous_gray = actual_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    
    
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    time.sleep(0.1)

captura.release()
cv2.destroyAllWindows()

In [ ]:
captura.release()

## Dense Optical flow

In [6]:
%reset -f
%matplotlib inline

import cv2 as cv2
import numpy as np
import matplotlib.pyplot as plt

import time

captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)

color = np.random.randint(0,255,(100,3))

ret, previous = captura.read()
previous = cv2.flip(previous,1)

previous_gray = cv2.cvtColor(previous, cv2.COLOR_RGB2GRAY)
hsv = np.zeros_like(previous)

w = previous.shape[0]
h = previous.shape[1]
ratio = w/h

fomv = 0

while(1):
    
    ret, actual = captura.read()
    actual = cv2.flip(actual,1)
    actual_gray = cv2.cvtColor(actual, cv2.COLOR_RGB2GRAY)
    
    # Calcula Fluxo Otico
    l = 120 # tamanho do lado da janela no centro, quadrada
    previous_gray_center = previous_gray[int((actual.shape[0]/2)-(l/2)+1):int((actual.shape[0]/2)+(l/2)+1), int((actual.shape[1]/2)-(l/2)+1):int((actual.shape[1]/2)+(l/2)+1)]#slicing para pegar o centro da imagem
    actual_gray_center = actual_gray[int((actual.shape[0]/2)-(l/2)+1):int((actual.shape[0]/2)+(l/2)+1), int((actual.shape[1]/2)-(l/2)+1):int((actual.shape[1]/2)+(l/2)+1)]
    flow = cv2.calcOpticalFlowFarneback(previous_gray_center, actual_gray_center,None, 0.5, 3, 15, 3, 5, 1.2, 0)       
    
    xmean = (np.mean(flow[:,:,0]))
    ymean = (np.mean(flow[:,:,1]))
    fom = np.array([xmean, ymean])
    fom *= -1
    fomv += fom
         
    
    framebw = cv2.cvtColor(actual, cv2.COLOR_RGB2GRAY)
    rows, cols = framebw.shape
    
    tx = (int(fomv[0]))
    ty = (int(fomv[1]))
    
    M = np.float32([[1,0,tx],
                    [0,1,ty]])
    
    framet = cv2.warpAffine(actual, M,(cols,rows))

    #scaling
    #dimensoes imagem cortada
    wt = w - abs(tx)
    ht = h - abs(ty)
    
    ratio_cut = wt/ht
    
    x_cut = 0
    y_cut = 0
    
    wtc = wt
    htc = ht
    
    if ratio > ratio_cut:
        #corta em y
        htc = int((h*wt)/w)
        y_cut = h - htc# - ty
    else:
        #corta em x
        wtc = int((ht*w)/h)
        x_cut = w - wtc# - tx
        
    new_ratio_cut = wtc/htc
    
    origin_x_cut = w - wtc - abs(tx) #tx - x_cut
    origin_y_cut = h - htc - abs(ty) #ty - y_cut
    
#     print(new_ratio_cut, ratio, tx, ty)
    framet_cut  = framet[origin_x_cut:(origin_x_cut + wtc), origin_y_cut:(origin_y_cut + htc)]
    
    
    M2 = np.float32([[1,0,int(origin_x_cut/2)],
                    [0,1,int(origin_y_cut/2)]])
    
    rows2, cols2, a = framet_cut.shape
    framet_cut = cv2.warpAffine(actual, M2,(cols2,rows2))

    frame_ok = cv2.resize(framet_cut,(h,w), interpolation = cv2.INTER_CUBIC)
    
#     txty = ("x" + str(tx) + "y" + str(ty))
#     font = cv2.FONT_HERSHEY_SIMPLEX
#     cv2.putText(framet,txty,(10,400), font, 1,(255,0,0),2,cv2.LINE_AA)
        
        
    cv2.imshow("Video", frame_ok)
#     cv2.imshow("centro", actual_gray_center)
    
    # Atualiza a imagem anterior com a imagem atual e copia os pontos.
    previous_gray = actual_gray
        
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    time.sleep(0.1)

captura.release()
cv2.destroyAllWindows()

In [5]:
captura.release()
